In [40]:
import pystan
import util
import pandas as pd
import numpy as np

In [63]:
player = 'Bubba Watson'
year = 2016
sqlTxt = '''
select name, permanent_tournament_id, scores.sg_tee as tee, scores.sg_approach as approach, 
       scores.sg_around as around, scores.sg_putting as putting from scores, players 
    where year = %s 
      and name like %s 
      and scores.player_id = players.player_id
      and scores.sg_putting is not null
      order by date;
'''
players_df = util.pd_from_sql(sqlTxt, [year, player])
y = [players_df[sg].as_matrix() for sg in['tee', 'approach', 'around', 'tee'] ]
sqlTxt = '''
select * from stan_sg_tournaments;
'''
tournaments_df = util.pd_from_sql(sqlTxt)
players_df

,name,permanent_tournament_id,tee,approach,around,putting
0,Bubba Watson,16,3.716,0.727,1.569,-2.134
1,Bubba Watson,16,4.233,2.264,0.480,-2.040
2,Bubba Watson,16,3.721,-0.834,-0.391,-2.799
3,Bubba Watson,16,4.268,1.240,0.470,0.017
4,Bubba Watson,3,2.351,1.247,0.018,-1.456
5,Bubba Watson,3,0.752,2.393,0.297,-1.108
6,Bubba Watson,3,-0.241,-0.067,-1.094,-0.045
7,Bubba Watson,3,2.269,1.435,1.076,0.726
8,Bubba Watson,5,0.647,-3.341,-1.716,0.801
9,Bubba Watson,7,0.910,2.265,-0.651,2.735


In [65]:
# 

code = '''
data {
    int<lower=1> N_SEG;
    int<lower=2> N;
    matrix[4,N] y;
    int<lower=2> N_PRED;
    real s_base;
}
parameters {
    matrix[4,N_SEG] alpha;
    vector[4] beta;
    vector<lower=0>[4] sigma;
    matrix[4,N_PRED] y_tilda;
} 
model {
    int m;
    for (k in 1:4) {
        for (n in 2:N) {
            m = (n-1)*N_SEG/N + 1;
            y[k][n] ~ normal(alpha[k][m]+ beta[k] * y[k][n-1], sigma[k]);
        }
    }
    for (j in 1:4) {
        y_tilda[j][1] ~ normal(alpha[j][N_SEG]+ beta[j] * y[j][N], sigma[j]);
        for (i in 2:N_PRED) {
            y_tilda[j][i] ~ normal(alpha[j][N_SEG] + beta[j]* y_tilda[j][i-1], sigma[j]);
        }
    }
}
generated quantities {
      vector[N_PRED] score_predict;
      for (l in 1:N_PRED) {
        score_predict[l] = s_base - y_tilda[1][l] - y_tilda[2][l] - y_tilda[3][l] - y_tilda[4][l];
    }
}

'''
data = {
    'N': len (y[0]),
    'y': y,
    'N_SEG' : 2,
    'N_PRED' :4, 
    's_base' : 71.571
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_5ba58b23c8ed993b96402931dd3cdf47 NOW.


Inference for Stan model: anon_model_5ba58b23c8ed993b96402931dd3cdf47.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

                   mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[0,0]         1.01  8.7e-3   0.32   0.41    0.8    1.0   1.23   1.66 1349.0    1.0
alpha[1,0]         0.29  8.4e-3   0.35   -0.4   0.05   0.29   0.52   0.96 1747.0    1.0
alpha[2,0]        -0.23  5.1e-3   0.23  -0.66  -0.38  -0.24  -0.08   0.21 2000.0    1.0
alpha[3,0]         1.02  7.7e-3   0.31   0.42   0.81   1.01   1.22   1.63 1650.0    1.0
alpha[0,1]         0.97  7.6e-3    0.3    0.4   0.77   0.98   1.17   1.55 1516.0    1.0
alpha[1,1]         0.32  8.4e-3   0.34  -0.37   0.09   0.31   0.54   0.98 1647.0    1.0
alpha[2,1]         0.13  4.9e-3   0.22   -0.3  -0.02   0.12   0.27   0.58 2000.0    1.0
alpha[3,1]         0.96  7.0e-3   0.28   0.42   0.78   0.96   1.15   1.52 1658.0    1.0
beta[0]            0.

In [64]:
print fit

Inference for Stan model: anon_model_5ba58b23c8ed993b96402931dd3cdf47.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

                   mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha[0,0]         0.81  5.3e-3   0.24   0.34   0.65    0.8   0.96   1.27 2000.0    1.0
alpha[1,0]         0.78  7.5e-3   0.34   0.12   0.56   0.78    1.0   1.44 2000.0    1.0
alpha[2,0]         0.43  4.0e-3   0.18   0.07   0.31   0.43   0.55   0.79 2000.0    1.0
alpha[3,0]         0.81  5.1e-3   0.23   0.37   0.66    0.8   0.96   1.25 2000.0    1.0
alpha[0,1]         0.49  4.6e-3    0.2   0.07   0.36   0.49   0.62   0.88 2000.0    1.0
alpha[1,1]         0.37  7.0e-3   0.31  -0.22   0.15   0.37   0.59   1.01 2000.0    1.0
alpha[2,1]         0.32  4.0e-3   0.18  -0.01    0.2   0.32   0.44   0.69 2000.0    1.0
alpha[3,1]         0.49  4.7e-3   0.21   0.07   0.35   0.49   0.63   0.91 2000.0    1.0
beta[0]            0.

In [ ]:

code = '''
data {
    int<lower=1> T;
    real y[T];
}
parameters {
    real mu;
    vector<lower = -1, upper = 1>[2] phi;
    vector<lower = -1, upper = 1>[2] theta;
    real<lower=0> sigma;
} 
model {
    vector[T] nu;
    vector[T] err;
    nu[1] = mu + phi[1] * mu;
    err[1] = y[1] - nu[1];
    nu[2] = mu + phi[1]*y[1] + theta[1]*err[1];
    err[2] = y[2] - nu[2];
    for (t in 3:T) {
        nu[t] = mu + phi[1] * y[t-1] + phi[2]*y[t-2] 
              + theta[1] * err[t-1] + theta[2] * err[t-2];
        err[t] = y[t] - nu[t];
    }
    mu ~ normal(0, 10);
    phi ~ normal(0, 2);
    theta ~ normal(0, 2);
    sigma ~ cauchy(0, 5);
    err ~ normal(0, sigma);
}
'''
y =  [ 0.596,  2.127,  3.093,  0.423, -1.508,  3.28 , -5.571,  2.623,
        1.335, -1.234, -0.364,  0.081, -1.076, -0.791,  1.715,  1.266,
        0.566, -1.333,  1.871,  2.429, -2.064, -0.336,  1.407,  2.787,
        2.743, -3.073,  1.335,  1.163,  2.982,  4.317,  2.913,  0.432,
       -3.238,  1.785,  2.766,  2.252,  1.024,  2.167, -0.334, -0.106,
        1.959, -0.784,  0.387,  2.816, -0.762, -1.897,  1.99 , -0.965,
        2.714,  1.237,  0.701,  3.814, -1.459,  1.021,  4.114,  0.038,
       -1.391,  2.91 ]
data = {
    'T': len (y),
    'y': y
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

In [14]:

code = '''
data {
    int<lower=1> T;
    real y[T];
}
parameters {
    real mu;
    vector<lower = -1, upper = 1>[2] phi;
    vector<lower = -1, upper = 1>[2] theta;
    real<lower=0> sigma;
    real y_tilda;
} 
model {
    vector[T] nu;
    vector[T] err;
    nu[1] = mu + phi[1] * mu;
    err[1] = y[1] - nu[1];
    nu[2] = mu + phi[1]*y[1] + theta[1]*err[1];
    err[2] = y[2] - nu[2];
    for (t in 3:T) {
        nu[t] = mu + phi[1] * y[t-1] + phi[2]*y[t-2] 
              + theta[1] * err[t-1] + theta[2] * err[t-2];
        err[t] = y[t] - nu[t];
    }
    mu ~ normal(0, 10);
    phi ~ normal(0, 2);
    theta ~ normal(0, 2);
    sigma ~ cauchy(0, 5);
    err ~ normal(0, sigma);
    y_tilda ~ normal(mu + y[T]*phi[1]+y[T-1]*phi[2] + theta[1]*err[T] + theta[2]*err[T-1], sigma);
}

'''
y =  [ 2.352,  2.619,  3.292,  2.657,  0.739,  1.115,  0.148,  0.572,
        0.859,  1.081,  1.104,  0.952, -1.324, -0.998,  1.124, -0.442,
        0.824, -0.825, -2.64 , -0.563,  1.129,  1.047,  0.943,  0.781,
       -0.933,  0.124,  0.341,  1.1  , -0.282,  1.173,  0.089,  1.544,
        0.026, -0.855,  1.448,  0.426, -0.245,  1.346, -0.36 , -0.328,
        0.482,  0.614,  0.572,  0.737, -0.097,  1.668, -0.012,  0.97 ,
       -0.898,  0.159,  1.543, -0.297,  1.257,  0.761,  0.725,  0.629,
        1.871,  0.061]
data = {
    'T': len (y),
    'y': y
}

fit = pystan.stan(model_code=code, data=data, iter=1000, chains=4)
# params = fs = fit.summary()['summary']
print fit

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_6abda944bb0b6acb21d3626f5be8dfef NOW.


Inference for Stan model: anon_model_6abda944bb0b6acb21d3626f5be8dfef.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu         0.61  9.2e-3   0.26   0.17   0.44    0.6   0.76   1.16  766.0    1.0
phi[0]     0.33    0.02   0.34  -0.46   0.14   0.41   0.57   0.85  295.0   1.02
phi[1]    -0.45    0.02   0.32   -0.9  -0.67  -0.51  -0.28   0.33  262.0   1.03
theta[0]   -0.1    0.02    0.3  -0.59  -0.31  -0.16   0.06   0.62  300.0   1.02
theta[1]    0.6    0.01   0.31  -0.18   0.46   0.68   0.84   0.97  450.0   1.02
sigma      1.01  2.8e-3    0.1   0.84   0.94    1.0   1.07   1.22 1223.0    1.0
y_tilda    0.62    0.03   1.08  -1.45  -0.09   0.63   1.31   2.82 1249.0    1.0
lp__     -33.79    0.09   2.05 -38.48 -34.94  -33.4 -32.31 -30.78  523.0    1.0

Samples were drawn using NUTS at Thu Aug 10 22:57:51 2017.
For each parameter, n_eff is a cr